In [1]:
using Unitful #https://painterqubits.github.io/Unitful.jl/stable/
#quantity * @u_str("unit abbreviation") 
using Symbolics #https://symbolics.juliasymbolics.org/dev/
#cite https://doi.org/10.48550/arXiv.2105.03949
using Latexify
using Test
#1 * @u_str("mA") is 1 milliamp
using CSV, DataFrames
#using Plots
using PlotlyJS
using Printf
using SymPy
using PDFIO
using Unzip
using Interpolations
using Plots
using TensorCast
#plotlyjs()
data_dir = "C:\\Cross-Section-Data\\EXFOR\\"

"C:\\Cross-Section-Data\\EXFOR\\"

Is it faster use a matrix or run more loops? Probably just to run more loops. 

In [17]:
file_path = data_dir * "neutrons\\092_U_238.c4"
file_as_vector = readlines(file_path)
max_line_length = maximum(length.(file_as_vector))
l = length(file_as_vector)
spacing_specifiers = [index for index in 1:length(file_as_vector) 
if check_line(file_as_vector[index], "# Prj")]
index = spacing_specifiers[1]
spacing_dict = make_spacing_dict(file_as_vector[index], file_as_vector[index + 1])
spacing_keys = [key for key in keys(spacing_dict)]
lines_of_data = tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
data_dict = Dict([])
filler = [data_dict[key] = [read_datum(file_as_vector[line_num][spacing_dict[key]])
            for line_num in index + 2: index + 1 + lines_of_data] for key in spacing_keys]   
data_dict

Dict{Any, Any} with 18 entries:
  "Targ"       => [92238.0, 92238.0, 92238.0, 92238.0, 92238.0, 92238.0, 92238.…
  "Entry"      => SubString{String}["V0015", "V0015", "V0015", "V0015", "V0015"…
  "dCos/LO"    => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "I78"        => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "dEnergy"    => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "dELV/HL"    => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "M"          => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "Refer (YY)" => SubString{String}["F.Manero,ET.AL. (72)", "F.Manero,ET.AL. (7…
  "Prj"        => [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.…
  "Energy"     => Any[1.25e6, 1.275e6, 1.3e6, 1.325e6, 1.35e6, 1.375e6, 1.4e6, …
  "Cos/LO"     => SubString{String}["", "", "", "", "", "", "", "", "", ""  …  …
  "ELV/HL"     => SubString{String}["", "", "", "", "", "", "", "", "", ""  …

In [14]:
dataset_df = DataFrame(data_dict)

,Cos/LO,Data,ELV/HL,Energy,Entry,I78,M,MF
,SubStrin…,Float64,SubStrin…,Any,SubStrin…,SubStrin…,SubStrin…,Float64
1,,2.5863,,1.25e6,V0015,,,1.0
2,,2.587,,1.275e6,V0015,,,1.0
3,,2.5878,,1.3e6,V0015,,,1.0
4,,2.5886,,1.325e6,V0015,,,1.0
5,,2.5896,,1.35e6,V0015,,,1.0
6,,2.5906,,1.375e6,V0015,,,1.0
7,,2.5917,,1.4e6,V0015,,,1.0
8,,2.5929,,1.425e6,V0015,,,1.0
9,,2.5941,,1.45e6,V0015,,,1.0


#Design

    Looping check_line() through each line in the file detects where each dataset begins. 

    Reading the line above each output of check_line() gives the number of rows in each dataset.

    make_spacing_dict() gets the names of the columns and the instructions for how to read each dataset.

    read_dataset() reads each and constructs a dataframe from each dataset.
    
    combine_datasets() creates vertically concatenates each dataset.

    read_exfor_file() runs all of the above in order to return a single DataFrame of a given file path.
     

In [7]:
function check_line(line, start)
    if length(line) > length(start) - 1
        return (line[1:length(start)] == start)
    end
    return false
end

function make_spacing_dict(line1, line2)
    spacing_ends = [collect(out)[1] for out in findall(">", line2)]
    spacing_starts = append!([1], [collect(out)[1] for out in findall("<", line2)])
    #Find where some of the spacings begin and end based on the arrows
    if length(spacing_ends) != length(spacing_starts)
        print("Error: improper formatting")
        return 
    end
    indices_with_gaps = [index for index in  1:length(spacing_ends)-1 if 
                    spacing_ends[index] != spacing_starts[index+1]-1]
    #Some of the spacings are instead denoted by the letter o instead of arrows
    missing_spacings = [spacing_ends[index]+1:spacing_starts[index+1]-1 for index in indices_with_gaps]
    spacings = append!(missing_spacings, 
    [spacing_starts[i]:spacing_ends[i] for i in 1:length(spacing_ends)])
    spacing_dict = Dict([])
    #Make a dictionary where the keys are name for each column and the values are the indices of the columns
    spacing_names = [spacing_dict[strip(line1[spacing], [' ', '#'])] = spacing for spacing in spacings]
    return spacing_dict
end

function read_datum(datum)
    #Reads a single datum from a line of data
    datum = strip(datum, [' '])
    out = tryparse(Float64, datum)
    if out == nothing
        return datum 
    end
    return out
end

function read_dataset(spacing_specifier, file_as_array)
    index = spacing_specifier
    spacing_dict = make_spacing_dict(file_as_vector[index], file_as_vector[index + 1])
    lines_of_data = tryparse(Int64, split(file_as_vector[index - 1], [' '])[end])
    data_dict = Dict([])
    filler = [data_dict[key] = [read_datum(file_as_vector[line_num][spacing_dict[key]])
            for line_num in index + 2: index + 1 + lines_of_data] for key in keys(spacing_dict)]    
    return data_dict, lines_of_data
end

read_dataset (generic function with 2 methods)

In [5]:
a = Dict(["dog" => 1, "cat" => 2, "fish" => 3])
a_keys = [key for key in keys(a)]
[a[key] for key in a_keys]

3-element Vector{Int64}:
 3
 2
 1

It it faster to use 1 or loop for $n$ list comprehensions? 

In [16]:
data_dir = "C:\\Cross-Section-Data\\EXFOR\\"
file_path = data_dir * "gammas\\001_H_001.c4"
file_as_array = split(open(f->read(f, String), file_path), "\n")
function read_exfor_file(file_path)
    #Reads an EXFOR file and returns a dictionary of data
    current_row = 1
    file_as_vector = readlines(file_path)
    max_line_length = maximum(length.(file_as_vector))
    l = length(file_as_vector)
    file_as_matrix =  Matrix{Char}(undef, l, max_line_length)
    file_as_nested_vector = [file_as_matrix[i,:] = [c for c in 
                            rpad(file_as_vector[i], max_line_length, " ")] 
                            for i in 1:length(file_as_vector)]
    #Split the file into lines
    #Find the line that starts with "Data"
    #==
    for index in 1:length(file_as_array)
        if check_line(file_as_array[index], "# Prj")
            rows_in_dataset = tryparse(Int64, split(file_as_array[index - 1], [' '])[end])
            spacing_dict = make_spacing_dict(file_as_array[index], file_as_array[index + 1])
            data_dict = Dict([])
            filler = [data_dict[key] =  [read_datum(
                    file_as_matrix[index + 2 : index + 1 + rows_in_dataset,spacing_dict[key]])
                    for key in keys(spacing_dict)]]    
        end
    end
    ==#
    spacing_specifiers = [index for index in 1:length(file_as_array) 
                    if check_line(file_as_nested_vector[index], "# Prj")]
    if spacing_specifiers == []
        print("Error: no data found")
        return
    end
    rows_in_each_dataset = [tryparse(Int64, split(file_as_array[index - 1], [' '])[end])
                    for index in spacing_specifiers]
    bottom_of_each_dataset = [sum(rows_in_each_dataset[1:index])
                    for index in 1:length(rows_in_each_dataset)-1]
    #Each dataset should have the same column names
    spacing_dict = make_spacing_dict(file_as_array[spacing_specifiers[1]], 
                    file_as_array[spacing_specifiers[1] + 1])
    spacing_dict_keys = [key for key in keys(spacing_dict)]
    #make an empty dataframe to fill up with the data
    df = DataFrame([Vector{Union{Missing, Float64, String}}(missing, bottom_of_each_dataset[end]) 
                    for _ in 1:length(spacing_dict_keys)], spacing_dict_keys)
    #Fill up the dataframe with the data
    out = [df[row] = 
    #data_list consists of the bottom of each dataset and the dataset stored as a dictionary 
    return combined_df
end
spacing_specifiers = [index for index in 1:length(file_as_array) 
                    if check_line(file_as_array[index], "# Prj")]
rows_in_each_dataset = [tryparse(Int64, 
                    split(file_as_array[index - 1], [' '])[end])
                    for index in spacing_specifiers]

13-element Vector{Int64}:
  8
  8
 57
 62
  4
  5
  4
  4
 18
 12
 14
  8
 21

In [16]:
bottom_of_each_dataset = [2, 10]
spacing_dict_keys = ["Energy", "Cross Section"]
df = DataFrame([Vector{Union{Missing, Float64, String}}(missing, bottom_of_each_dataset[end]) 
                    for _ in 1:length(spacing_dict_keys)], spacing_dict_keys)
df[1,1] = "dog"
df

,Energy,Cross Section
,Union…?,Union…?
1,dog,missing
2,missing,missing
3,missing,missing
4,missing,missing
5,missing,missing
6,missing,missing
7,missing,missing
8,missing,missing
9,missing,missing


In [19]:
bottom_of_each_dataset = [sum(rows_in_each_dataset[1:index])
                        for index in 1:length(rows_in_each_dataset)-1]

12-element Vector{Int64}:
   8
  16
  73
 135
 139
 144
 148
 152
 170
 182
 196
 204

In [4]:
df_list = [DataFrame(read_data(spacing_specifier, file_as_array))
            for spacing_specifier in spacing_specifiers]

13-element Vector{DataFrame}:
 8×18 DataFrame
 Row │ Cos/LO     Data     ELV/HL     Energy     Entry      I78        M       ⋯
     │ SubStrin…  Float64  SubStrin…  SubStrin…  SubStrin…  SubStrin…  SubStri ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │            0.01876             3.3000+8   K2066                         ⋯
   2 │            0.01906             4.1000+8   K2066
   3 │            0.01925             5.2000+8   K2066
   4 │            0.01932             6.0000+8   K2066
   5 │            0.01932             6.8000+8   K2066                         ⋯
   6 │            0.01952             7.5000+8   K2066
   7 │            0.01961             8.3000+8   K2066
   8 │            0.01972             9.1000+8   K2066
                                                              12 columns omitted
 8×18 DataFrame
 Row │ Cos/LO     Data       ELV/HL     Energy     Entry      I78        M     ⋯
     │ SubStrin…  SubStrin…  SubStrin…  S

In [6]:
list_of_df_rows = append!([0],[[s for s in size(df)][1] for df in df_list])
n_rows = sum(list_of_df_rows)
n_cols = size(df_list[1])[2]
combined_df = DataFrame([Vector{Union{Missing, Float64}}(missing, n_rows) 
                for _ in 1:n_cols], names(df_list[1]))

,Cos/LO,Data,ELV/HL,Energy,Entry,I78,M,MF
,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,missing,missing,missing,missing,missing,missing,missing,missing
2,missing,missing,missing,missing,missing,missing,missing,missing
3,missing,missing,missing,missing,missing,missing,missing,missing
4,missing,missing,missing,missing,missing,missing,missing,missing
5,missing,missing,missing,missing,missing,missing,missing,missing
6,missing,missing,missing,missing,missing,missing,missing,missing
7,missing,missing,missing,missing,missing,missing,missing,missing
8,missing,missing,missing,missing,missing,missing,missing,missing
9,missing,missing,missing,missing,missing,missing,missing,missing


In [26]:
list_of_df_rows = append([0], list_of_df_rows(df_list))

,MT,PXC,Prj,Refer (YY),Sub,Targ,dCos/LO
,Float64,SubStrin…,Float64,SubStrin…,Float64,Float64,SubStrin…
1,1.0,,0.0,"S.Homma,ET.AL. (74)",7.0,1001.0,
2,1.0,,0.0,"S.Homma,ET.AL. (74)",7.0,1001.0,
3,1.0,,0.0,"S.Homma,ET.AL. (74)",7.0,1001.0,
4,1.0,,0.0,"S.Homma,ET.AL. (74)",7.0,1001.0,
5,1.0,,0.0,"S.Homma,ET.AL. (74)",7.0,1001.0,
6,1.0,,0.0,"S.Homma,ET.AL. (74)",7.0,1001.0,
7,1.0,,0.0,"S.Homma,ET.AL. (74)",7.0,1001.0,
8,1.0,,0.0,"S.Homma,ET.AL. (74)",7.0,1001.0,
9,2.0,C,0.0,"A.Hunger,ET.AL. (97)",3.0,1001.0,


In [56]:
n_rows

n_rows (generic function with 1 method)

In [57]:
DataFrame([Vector{Union{Missing, Float64}}(missing, n_rows(df_list)) for _ in 1:n_cols], names(df_list[1]))

,Cos/LO,Data,ELV/HL,Energy,Entry,I78,M,MF
,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?,Float64?
1,missing,missing,missing,missing,missing,missing,missing,missing
2,missing,missing,missing,missing,missing,missing,missing,missing
3,missing,missing,missing,missing,missing,missing,missing,missing
4,missing,missing,missing,missing,missing,missing,missing,missing
5,missing,missing,missing,missing,missing,missing,missing,missing
6,missing,missing,missing,missing,missing,missing,missing,missing
7,missing,missing,missing,missing,missing,missing,missing,missing
8,missing,missing,missing,missing,missing,missing,missing,missing
9,missing,missing,missing,missing,missing,missing,missing,missing
